# Convert a Trove list into a CSV file

This notebook converts [Trove lists](https://trove.nla.gov.au/list/result?q=) into CSV files (spreadsheets). Separate CSV files are created for newspaper articles and works from Trove's other zones. The OCRd text of each newspaper article is also saved in a individual text file.

* To run a cell just hit **Shift+Enter** or click the play icon
* To edit a cell, just click on it

## Add your values to these two cells

This is the only section that you'll need to edit. Paste your API key and list id in the cells below as indicated.

If necessary, follow the instructions in the Trove Help to [obtain your own Trove API Key](http://help.nla.gov.au/trove/building-with-trove/api).

The list id is the number in the url of your Trove list. So [the list](https://trove.nla.gov.au/list?id=83774) with this url `https://trove.nla.gov.au/list?id=83774` has an id of `83774`.

In [ ]:
# Paste you API key between the quotes, and then run the cell
api_key = ''
print('Your API key is: {}'.format(api_key))

In [ ]:
# Paste your list id between the quotes, and then run the cell
list_id = '83774'
# If you don't want to save all the OCRd text, change True to False below
save_texts = True

## Set things up

Run the cell below to load the necessary libraries and set up some directories to store the results.

In [ ]:
import requests
from requests.exceptions import HTTPError, Timeout
import pandas as pd
import os
import re
import shutil
from IPython.core.display import display, HTML

# Make sure data directory exists
data_dir = '../../data/Trove/converted-lists'
os.makedirs(data_dir, exist_ok=True)

list_dir = '{}/{}'.format(data_dir, list_id)
os.makedirs('{}/texts'.format(list_dir), exist_ok=True)

works = []
articles = []

## Define some functions

Run the cell below to set up all the functions we'll need for the conversion.

In [ ]:
def listify(value):
    '''
    Sometimes values can be lists and sometimes not.
    Turn them all into lists to make life easier.
    '''
    if isinstance(value, (str, int)):
        try:
            value = str(value)
        except ValueError:
            pass
        value = [value]
    return value

def get_url(identifiers, linktype):
    '''
    Loop through the identifiers to find the request url.
    '''
    url = ''
    for identifier in identifiers:
        if identifier['linktype'] == linktype:
            url = identifier['value']
            break
    return url

def save_as_csv(data, data_type):
    df = pd.DataFrame(data)
    df.to_csv('{}/{}-{}.csv'.format(list_dir, list_id, data_type), index=False)
    
def make_filename(article):
    date = article['date']
    date = date.replace('-', '')
    newspaper_id = article['title']['id']
    article_id = article['id']
    return '{}/texts/{}-{}-{}'.format(list_dir, date, newspaper_id, article_id)

def get_list():
    api_url = 'http://api.trove.nla.gov.au/list/{}?encoding=json&reclevel=full&include=listItems&key={}'
    list_url = api_url.format(list_id, api_key)
    response = requests.get(list_url)
    return response.json()

def get_article(id):
    article_api_url = 'http://api.trove.nla.gov.au/newspaper/{}/?encoding=json&reclevel=full&include=articletext&key={}'.format(id, api_key)
    response = requests.get(article_api_url)
    return response.json()

def harvest_list():
    data = get_list()
    print('Saving items...')
    for item in data['list'][0]['listItem']:
        for zone, record in item.items():
            if zone == 'work':
                work = {
                    'id': record.get('id', ''),
                    'title': record.get('title', ''),
                    'type': '|'.join(listify(record.get('type', ''))),
                    'issued': '|'.join(listify(record.get('issued', ''))),
                    'contributor': '|'.join(listify(record.get('contributor', ''))),
                    'trove_url': record.get('troveUrl', ''),
                    'fulltext_url': get_url(record.get('identifier', ''), 'fulltext'),
                    'thumbnail_url': get_url(record.get('identifier', ''), 'thumbnail')
                }
                works.append(work)
            elif zone == 'article':
                article = {
                    'id': record.get('id'),
                    'title': record.get('heading', ''),
                    'category': record.get('category', ''),
                    'date': record.get('date', ''),
                    'newspaper_id': record.get('title', {}).get('id'),
                    'newspaper_title': record.get('title', {}).get('value'),
                    'page': record.get('page', ''),
                    'page_sequence': record.get('pageSequence', ''),
                    'trove_url': 'http://nla.gov.au/nla.news-article{}'.format(record.get('id', ''))
                }
                full_details = get_article(record.get('id'))
                article['words'] = full_details['article'].get('wordCount', '')
                article['illustrated'] = full_details['article'].get('illustrated', '')
                article['corrections'] = full_details['article'].get('correctionCount', '')
                if 'trovePageUrl' in full_details['article']:
                    page_id = re.search(r'page\/(\d+)', full_details['article']['trovePageUrl']).group(1)
                    article['page_url'] = 'http://trove.nla.gov.au/newspaper/page/{}'.format(page_id)
                else:
                    article['page_url'] = ''
                if save_texts:
                    text = full_details['article'].get('articleText')
                    if text:
                        text_filename = make_filename(record)
                        text = re.sub('<[^<]+?>', '', text)
                        text = re.sub("\s\s+", " ", text)
                        text_file = '{}.txt'.format(text_filename)
                        with open(text_file, 'wb') as text_output:
                            text_output.write(text.encode('utf-8'))
                articles.append(article)
    print('Converting to CSV...')
    if articles:
        save_as_csv(articles, 'articles')
    if works:
        save_as_csv(works, 'works')
    print('Finished!')

## Let's do it!

Run the cell below to start the conversion.

In [ ]:
harvest_list()

## View the results

You can browse the harvested files in the [data directory](../../data/Trove/converted-lists).

Run the cells below for a preview of the CSV files.

In [ ]:
# Preview newspaper articles CSV
df_articles = pd.DataFrame(articles)
df_articles

In [ ]:
# Preview works CSV
df_works = pd.DataFrame(works)
df_works

## Download the results

Run the cell below to zip up all the harvested files and create a download link.

In [ ]:
shutil.make_archive(list_dir, 'zip', list_dir)
display(HTML('<a target="_blank" href="{}/{}.zip">Download your harvest</a>'.format(data_dir, list_id)))